In [ ]:
%load_ext autoreload
%autoreload 2 
%matplotlib inline

In [ ]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]=""

import numpy as np
import tensorflow as tf
from shutil import copy

import matplotlib.pyplot as plt
import matplotlib

In [ ]:
pathfig = 'figures/'
os.makedirs(pathfig, exist_ok=True)

In [ ]:
def get_event_data(event_files, tag, selec_it=None):
    data = []
    it = []
    if selec_it:
        selec_it = set(selec_it)
    for path_event in event_files:
        try:
            for e in tf.train.summary_iterator(path_event):
                for v in e.summary.value:
                    if tag in v.tag:
                        if selec_it is None or e.step in selec_it:
                            data.append(v.simple_value)
                            it.append(e.step)
        except:
            print('Warning corrupted file')
    return np.array(data), np.array(it)


def get_event_files(summary_dir):
    # Getting the event file
    event_files = []
    for filename in os.listdir(summary_dir):
        if 'events.out.tfevents' in filename:
            event_files.append(os.path.join(summary_dir, filename))
    # if len(event_files)>1:
    #     raise ValueError('Multiple event files')
    if len(event_files) == 0:
        raise ValueError('No event files')
    return event_files


In [ ]:
summary_dir = 'saved_results/commands_md64_8k_summary/'
event_files = get_event_files(summary_dir)

In [ ]:
selec_it = None
# Getting data
duality_gap, it = get_event_data(event_files, 'duality/gap_1', selec_it) 
duality_minmax, itt = get_event_data(event_files, 'duality/minmax_1', selec_it) 
np.testing.assert_almost_equal(it,itt)
duality_maxmin, itt = get_event_data(event_files, 'duality/maxmin_1', selec_it) 
np.testing.assert_almost_equal(it,itt)
consistency, itt = get_event_data(event_files, 'Gen/Reg', selec_it) 
np.testing.assert_almost_equal(it,itt)

# Reodering
indexes = np.argsort(it)
it = it[indexes]
duality_gap = duality_gap[indexes]
duality_maxmin = duality_maxmin[indexes]
duality_minmax = duality_minmax[indexes]
consistency = consistency[indexes]

In [ ]:
N = 16
win = np.ones([N])
win = win/np.sum(win)

duality_gap_smooth = np.convolve(duality_gap, win, mode='same')
duality_minmax_smooth = np.convolve(duality_minmax, win, mode='same')
consistency_smooth = np.convolve(consistency, win, mode='same')

select = np.logical_and(it>=20000, it<=80000)


In [ ]:
# plt.plot(it[select],duality_gap[select],color='b', alpha=0.2)
# plt.plot(it[select],duality_gap_smooth[select], label='DG', color='b')
# plt.plot(it[select],2.5*duality_minmax[select], color='orange', alpha=0.3)
# plt.plot(it[select],2.5*duality_minmax_smooth[select], label='MinMax value',  color='orange')
# plt.plot(it[select],60*consistency[select], color='green', alpha=0.3)
# plt.plot(it[select],60*consistency_smooth[select], label='Consistency measure', color='green')
# plt.legend()

In [ ]:
plt.rcParams['figure.figsize'] = (8, 3)  # (9, 4) for matplotlib notebook
matplotlib.rcParams.update({'font.size': 16})

# Plot the results
fig, ax1 = plt.subplots()

color = 'tab:blue'
color2 = 'tab:orange'
ax1.set_xlabel('Iteration')
ax1.set_ylabel('Gap')
ax1.plot(it[select],duality_gap[select]/3,color=color, alpha=0.2)
ax1.plot(it[select],duality_gap_smooth[select]/3, label='Dual gap /3', color=color)
ax1.plot(it[select],duality_minmax[select], color=color2, alpha=0.3)
ax1.plot(it[select],duality_minmax_smooth[select], color=color2, label='MinMax value')
# ax1.tick_params(axis='y', labelcolor=color)
ax1.legend()
ax1.set_ylim([0,7])
ax1.set_xlim([np.min(it[select]),np.max(it[select])])

ax3 = ax1.twinx()  # instantiate a second axes that shares the same x-axis

color = 'tab:green'
ax3.set_ylabel('Consistency score', color=color)  # we already handled the x-label with ax1
ax3.plot(it[select],consistency[select], color=color, alpha=0.3)
ax3.plot(it[select],consistency_smooth[select], color=color)
ax3.tick_params(axis='y', labelcolor=color)
# ax2.legend(loc=2)
ax3.set_ylim([0,0.3])

fig.tight_layout()  # otherwise the right y-label is slightly clipped
fig.savefig(os.path.join(pathfig, "tifgan_consistency.pdf"), bbox_inches='tight')